In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '../functions/')
from statistics_functions import *
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
import cartopy.crs as ccrs
import cartopy

In [2]:
base_path  = '/work/bk1377/a270230/'
path_ds    = base_path + 'lkf_datasets/weekly_data/'
path_ice   = base_path + 'interpolated_fesom_output/'

In [3]:
files = os.listdir(path_ds)
files.sort()
files.remove('.ipynb_checkpoints')
#files

# Analyzing the Weekly Data, Comparing different DoG Thresholds

In [ ]:
datasets, lkfs = get_lkf_data(path_ds, years)

In [4]:
years, dogs = [], []
for file in files:
    years += file.split('_')[1].split('.')[0],
    dogs  += file.split('_')[2][1:-4],
    
# get the lkf_data objects
datasets = []
for file in files:
    datasets += np.load(path_ds + file, allow_pickle=True)[0],
    
# get the lkf data in arrays for every timestep
lkfs = []
for lkf_data in datasets:
    lkfs += get_lkfs(lkf_data),
    
# calculate the total ice area (in km2) at every time step
ice_coverage, ice_coverage_weekly = get_ice_coverage(years, path_ice)

# get the number of lkfs at each timestep
n_lkfs = []
for lkfs_dataset in lkfs:
    n_lkfs += get_n_lkfs(lkfs_dataset),
    
# get the number of lkfs per ice covered area at every timestep
rho_lkfs = [[] for _ in dogs]
for year in range(len(dogs)):
    rho_lkfs[year] = np.array(n_lkfs[year]) / np.array(ice_coverage_weekly[year])

NameError: name 'get_lkfs' is not defined

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(11,4))

for ax, data, ylabel in zip(
    axs, [n_lkfs, rho_lkfs], ['no. LKFs', 'no. LKFs /100000km$^2$']
):
    for dog, data_i in zip(dogs, data):
        ax.plot(data_i, label=dog)
    ax.set_ylabel(ylabel)
    ax.set_xlabel('time /weeks')
    ax.grid()
axs[1].legend(bbox_to_anchor=(1,1))

fig.tight_layout()

In [ ]:
# get the length of every LKF as well as the average
# LKF length in pixels at every timestep
lkf_len, av_lkf_len, total_lkf_len = [], [], []
for lkfs_dataset in lkfs:
    a, b, c = get_lkf_len(lkfs_dataset)
    lkf_len       += a,
    av_lkf_len    += b,
    total_lkf_len += c,

In [ ]:
dogs

In [ ]:
i = 0
j = 1
d = 365

fig, axs = plt.subplots(1,2, figsize=(11,4))

for ax, data, mean, title in zip(
    axs, [lkf_len[i][:d], lkf_len[j][:d]], [av_lkf_len[i][:d], av_lkf_len[j][:d]], [dogs[i], dogs[j]]
):

    for timestep, lkf_len_timestep in enumerate(data):

        try:
            frequency = np.zeros(int(np.max(lkf_len_timestep)+1))
            for lkf_len_i in lkf_len_timestep:
                frequency[lkf_len_i] += 1

            f_norm = frequency/frequency.max()
        except:
            f_norm = 0

        ax.scatter(timestep * np.ones_like(lkf_len_timestep), lkf_len_timestep, s=5, c='k', alpha=f_norm)

    ax.plot(mean, 'b')
    ax.grid()
    ax.set_xlabel('time /weeks')
    ax.set_ylabel('LKF length /km')
    ax.set_ylim([0,2500])
    ax.set_title('dog = ' + title)

fig.tight_layout()

In [ ]:
plt.figure(figsize=(5,4))
for dog, lkf_len in zip(dogs, av_lkf_len):
    plt.plot(lkf_len, label=dog)
    
plt.grid()
plt.ylim([0,300])
plt.xlabel('time/ weeks')
plt.ylabel('Mean LKFs length /km')
plt.legend(bbox_to_anchor=(1,1))

# 